Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

T.b.d.


In [1]:
import logging
from collections import namedtuple
import random

## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

## Oversimplified match

In [6]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (pure_random, gabriele)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 1 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 1 -> <0 1 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <0 1 5 7 9 5 13 15 17 19 21>
DEBUG:root:status: After player 1 -> <0 0 5 7 9 5 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <0 0 5 7 9 5 13 15 17 19 14>
DEBUG:root:status: After player 1 -> <0 0 0 7 9 5 13 15 17 19 14>
DEBUG:root:status: After player 0 -> <0 0 0 7 9 5 13 15 17 19 8>
DEBUG:root:status: After player 1 -> <0 0 0 0 9 5 13 15 17 19 8>
DEBUG:root:status: After player 0 -> <0 0 0 0 9 5 13 12 17 19 8>
DEBUG:root:status: After player 1 -> <0 0 0 0 0 5 13 12 17 19 8>
DEBUG:root:status: After player 0 -> <0 0 0 0 0 5 13 12 9 19 8>
DEBUG:root:status: After player 1 -> <0 0 0 0 0 0 13 12 9 19 8>
DEBUG:root:status: After player 0 -> <0 0 0 0 0 0 13 2 9 19 8>
DEBUG:root:status: After player 1 -> <0 0 0 0 0 0 0 2 9 19 8>
DEBUG:root:status: Aft